In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Guided By Voices,Everyday,Clouds open dust and waves rise up to \r\nObs...
1,Leonard Cohen,Show Me The Place,"Show me the place, where you want your slave t..."
2,Chaka Khan,Dance Wit Me,Dance wit me ev'rybody dance wit me \r\nDance...
3,Neil Sedaka,No Vacancy,I'm livin' in a house on the other side of tow...
4,Jimmy Buffett,Manana,Jimmy Buffett \r\nShe said I can't go back to...
5,Poison,Unskinny Bop,What's got you so jumpy? \r\nWhy can't you si...
6,Depeche Mode,Never Let Me Down Again,I'm taking a ride \r\nWith my best friend \r...
7,Ellie Goulding,Hanging On,You know we can get away \r\nBecause I'm call...
8,Lucky Dube,It's Not Easy,I remember the day I called mama on the teleph...
9,Amy Grant,Prince Of Peace,Sometimes my life just don't make sense at all...


In [9]:
df['text'][0]

"Clouds open dust and waves rise up to  \r\nObscure God's very blue sky.  \r\nPeople skating ginger(ly), only endure  \r\nLines of shopping mothers grabbing the kids  \r\nPushing upto the front  \r\nRows of pretty little house and lawns  \r\nI want you to have a beautiful one.  \r\nMoving in circles out on the freeway  \r\nComing and going to everyday, everyday  \r\nMoving in circles out on the freeway  \r\nComing and going to everyday, everyday  \r\nGot a brand new car with hideaway lights  \r\nAnd a blue racing stripe  \r\nCosts a little extra but its worth it to me  \r\nWhen money burns through the exhaust pipe  \r\nI'd like to take you on a quick spin sometime  \r\nTo show you what its got  \r\nI can't believe you chose to work in the line  \r\nI'm so glad that I'm not  \r\nEveryday and everyway  \r\nEveryday and everyway  \r\nEveryday and everyway  \r\n  \r\n\r\n"

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.01078279, 0.00591198, ..., 0.        , 0.02820371,
       0.00242437])

In [24]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [20]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [22]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))